In [2]:
import math
from queue import PriorityQueue
import time

In [3]:
puzzle = 8
dim = int(math.sqrt(puzzle)) + 1
print(dim)

3


In [5]:
goal_state = []
co_ordinates = {}
for i in range(dim):
  temp = []
  for j in range(dim):
    val = i*dim+(j+1)
    temp.append(val)
    co_ord = [i, j]
    co_ordinates[val] = co_ord
  goal_state.append(temp)

last_val = goal_state[-1][-1]
goal_state[-1][-1] = 0
temp_co_ord = co_ordinates[last_val]
del co_ordinates[last_val]
co_ordinates[0] = temp_co_ord
print(last_val)
print(goal_state)
print(co_ordinates)

9
[[1, 2, 3], [4, 5, 6], [7, 8, 0]]
{1: [0, 0], 2: [0, 1], 3: [0, 2], 4: [1, 0], 5: [1, 1], 6: [1, 2], 7: [2, 0], 8: [2, 1], 0: [2, 2]}


In [9]:
def manhattan_distance(state):
  m_dist = 0
  for i in range(dim):
    for j in range(dim):
      val = state[i][j]
      if(val == 0):
        continue
      goal_co_ord = co_ordinates[val]
      i_goal = goal_co_ord[0]
      j_goal = goal_co_ord[1]
      m_dist += abs(i_goal - i) + abs(j_goal - j)
  return m_dist

In [10]:
state = [[7, 2, 4], [5, 0, 6], [8, 3, 1]]
dist = manhattan_distance(state)
print(dist)

14


In [13]:
def misplaced_tile(state):
  m_tile = 0
  for i in range(dim):
    for j in range(dim):
      val = goal_state[i][j]
      if(val == 0):
        continue
      if(val != state[i][j]):
        m_tile += 1
  return m_tile

In [14]:
state = [[7, 2, 4], [5, 0, 6], [8, 3, 1]]
dist = misplaced_tile(state)
print(dist)

6
